<a href="https://colab.research.google.com/github/No-Qubit-Left-Behind/Control-Engineering-in-TF/blob/master/TF_GRAPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Propagator

In [29]:
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import time
print(tf.__version__)

2.2.0-rc1


In [30]:
class PropagatorVL:
    def __init__(self, no_of_steps, delta_t):
        self.delta_t = delta_t
        self.duration = no_of_steps * delta_t

        self.x = tf.constant(
            [[0, 1], [1, 0]], dtype=tf.complex128
        )
        self.y = tf.constant(
            [[0 + 0j, 0 - 1j], [0 + 1j, 0 + 0j]], dtype=tf.complex128
        )
        
        """
            Van Loan generators for control robustness
        """
        xL = np.block([
            [self.x.numpy(), self.x.numpy()],
            [np.zeros((2, 2)), self.x.numpy()]          
        ])
        yL = np.block([
            [self.y.numpy(), self.y.numpy()],
            [np.zeros((2, 2)), self.y.numpy()]          
        ])

        self.generators =  tf.stack([
                                     tf.constant(xL, dtype=tf.complex128),
                                     tf.constant(yL, dtype=tf.complex128)
                                     ])

        self.ctrl_amplitudes = tf.Variable(
            tf.zeros([no_of_steps, 2], dtype=tf.float64), dtype=tf.float64
        )


        """
            self.contraction_array determines the neccessity for the extra
            matrix multiplication step in the recursive method self.propagate()
            when the intermediate computation array has length not divisible
            by 2
        """
        self.contraction_array = []
        contraction_array_length = int(np.floor(np.log2(no_of_steps)))
        temp_no_of_steps = no_of_steps
        for i in range(contraction_array_length):
            self.contraction_array.append(bool(np.mod(temp_no_of_steps, 2)))
            temp_no_of_steps = np.floor(temp_no_of_steps / 2)
  
    """
        exponentials() computes a vector matrix exponential after multiplying
        each self.ctrl_amplitudes row with a the vector of matrices in
        self.generators
    """
    def exponentials(self):
        regularized_amplitudes = 1 / np.sqrt(2) * tf.math.tanh(
            self.ctrl_amplitudes
        )

        exponents = tf.linalg.tensordot(
            tf.cast(regularized_amplitudes, dtype=tf.complex128),
            -2 * np.pi *(0 + 1j) * self.delta_t * self.generators, 1
        )
        return tf.linalg.expm(exponents)
    
    """
        propagate  computes the final propagator by recursively multiplying
        each odd element in the list of matrices with each even element --
        if the length of the array is not divisible by 2 an extra computation
        step is added
    """
    def propagate(self):
        step_exps = self.exponentials()
        for is_odd in self.contraction_array:
            if is_odd:
                odd_exp = step_exps[-1, :, :]
                step_exps = tf.linalg.matmul(
                    step_exps[1::2, :, :], step_exps[0:-1:2, :, :]
                )
                step_exps = tf.concat([
                    step_exps[0:-1, :, :],
                    [tf.linalg.matmul(odd_exp, step_exps[-1, :, :])]
                ], 0)
            else:
                step_exps = tf.linalg.matmul(
                    step_exps[1::2, :, :], step_exps[0::2, :, :]
                )
        return tf.squeeze(step_exps)

    """
        __call__ computes the final propagator fidelity squared with the
        identity operator
    """
    @tf.function
    def infidelity(self):
        propagator = self.propagate()
        tr = tf.linalg.trace(tf.linalg.matmul(self.x, propagator[0:2, 0:2]))
        """
            infidelity part in the target
        """
        infidelity = 1 - tf.math.real(tr * tf.math.conj(tr)) / (2 ** 2)
        """
            robustness term in the target
        """
        norm_squared = 1 / ((2 * np.pi * self.duration) ** 2) / 2 * (
            tf.math.real(
                tf.linalg.trace(
                    tf.linalg.matmul(
                        propagator[0:2, 3:4],
                        propagator[0:2, 3:4],
                        adjoint_b=True
                    )
                )
            )
        )

        return 0.5 * infidelity + 0.5 * norm_squared

propagatorVL = PropagatorVL(1000, 0.001)

optimizer = tf.keras.optimizers.Adam(0.01)

propagatorVL.ctrl_amplitudes.assign(
    tf.random.uniform([1000, 2], -1, 1, dtype=tf.float64)
)

def optimization_step():
    with tf.GradientTape() as tape:
        infidelity = propagatorVL.infidelity()
    gradients = tape.gradient(infidelity, [propagatorVL.ctrl_amplitudes])
    optimizer.apply_gradients(zip(gradients, [propagatorVL.ctrl_amplitudes]))
    return infidelity

steps = range(1000)
for step in steps:
    current_infidelity = optimization_step()
    print('step %2d: infidelity=%2.5f' %
          (step, current_infidelity))
    
propagatorVL.ctrl_amplitudes.numpy()

step  0: infidelity=0.49939
step  1: infidelity=0.49767
step  2: infidelity=0.49496
step  3: infidelity=0.49127
step  4: infidelity=0.48657
step  5: infidelity=0.48085
step  6: infidelity=0.47409
step  7: infidelity=0.46629
step  8: infidelity=0.45745
step  9: infidelity=0.44758
step 10: infidelity=0.43670
step 11: infidelity=0.42482
step 12: infidelity=0.41198
step 13: infidelity=0.39822
step 14: infidelity=0.38360
step 15: infidelity=0.36817
step 16: infidelity=0.35201
step 17: infidelity=0.33521
step 18: infidelity=0.31787
step 19: infidelity=0.30009
step 20: infidelity=0.28197
step 21: infidelity=0.26363
step 22: infidelity=0.24521
step 23: infidelity=0.22684
step 24: infidelity=0.20864
step 25: infidelity=0.19075
step 26: infidelity=0.17330
step 27: infidelity=0.15641
step 28: infidelity=0.14021
step 29: infidelity=0.12480
step 30: infidelity=0.11028
step 31: infidelity=0.09674
step 32: infidelity=0.08424
step 33: infidelity=0.07284
step 34: infidelity=0.06256
step 35: infidelity=

array([[ 0.58505743,  1.62318135],
       [ 0.16474558,  1.58980122],
       [ 1.57057091,  1.53872745],
       ...,
       [ 0.27915669, -2.0112761 ],
       [ 0.02295734, -2.08824812],
       [ 0.47861396, -1.65595605]])